In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap

In [13]:
# read in mask
barrow_mask = gpd.read_file("Barrow_mask.gpkg")
style_dict = {
    'fillColor': '#000000',
    'fillOpacity': 0.7,
    'stroke':False
}

# read in county data
county_permits = pd.read_csv('Barrow_geocoded_v2.csv')
county_res = county_permits[(county_permits['PROJ DESC']=='RESIDENTIAL-MULTI FAMILY') | (county_permits['PROJ DESC']=='RESIDENTIAL SINGLE FAMILY')]
county_comm = county_permits[county_permits['PROJ DESC']=='COMMERCIAL NEW BUILDING']

# make county permit heatmap data
residential_data = [[row['lat'], row['long']] for index, row in county_res.iterrows()]
commercial_data = [[row['lat'], row['long']] for index, row in county_comm.iterrows()]

# define map boundaries
min_lon, max_lon = -84.20, -83.20
min_lat, max_lat = 33.68, 34.28

m = folium.Map(
    location=[34.01, -83.70],
    tiles=None, 
    zoom_start=11.5,
    max_bounds=True,
    min_zoom=11,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon,
)

# add mask
polygon = folium.GeoJson(barrow_mask,control=False, style_function = lambda x: style_dict).add_to(m)

# Residential heatmap
residential_heatmap = HeatMap(
    residential_data, 
    overlay=True,
    show=True,
    name="Residential Permits - Heatmap"
).add_to(m)

# Add small blue semi-transparent circles for residential permits
feature_gp2 = folium.FeatureGroup(name="Residential Permits - Dots", show=False)
for index, row in county_res.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['long']],
        radius=7,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=f"{row['PROJ DESC']} - {row['LOCATION']}",
    ).add_to(feature_gp2)
    
feature_gp2.add_to(m)

# Commercial heatmap
commercial_heatmap = HeatMap(
    commercial_data, 
    overlay=True,
    show=False,
    name="Commercial Permits - Heatmap"
).add_to(m)

# Commercial dots
feature_gp1 = folium.FeatureGroup(name="Commercial Permits - Dots", show=False)
for index, row in county_comm.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['long']],
        radius=7,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        popup=f"{row['PROJ DESC']} - {row['LOCATION']}",
    ).add_to(feature_gp1)
feature_gp1.add_to(m)

# add basemaps
folium.raster_layers.TileLayer(tiles='cartodbpositron', name='Roads').add_to(m)
folium.raster_layers.TileLayer(
    tiles='http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}', 
    attr='&copy; <a href="https://www.google.com/maps" target="_blank">Google Maps</a> ',
    name='Satellite'
).add_to(m)

# Add layer control toggle
folium.LayerControl().add_to(m)

m.save("index.html")
m